## Deep Learning with pytorch

### Basics of torch and numpy

In [39]:
import torch
import numpy as np

a = torch.FloatTensor(1, 3, 2)
a
torch.zeros(2, 6)
a.zero_()
b = torch.FloatTensor([[1, 2, 3], [3, 2, 1]])
b
c = np.zeros((3, 3), dtype=np.int32)
d = torch.tensor((c))
c, d, np.concatenate((c, d))
## no dim vs 1 dim
no_dim = torch.tensor(3)
one_dim = torch.tensor([3]) # or
no_dim, one_dim, no_dim.shape, one_dim.shape


(tensor(3), tensor([3]), torch.Size([]), torch.Size([1]))

### Computation device types; cpu vs gpu vs mps

In [ ]:
a = torch.FloatTensor([2, 3])
# ca = a.to('cuda') # this mac doesn't have cuda but what could have been :)
ma = a.to('mps')
a, ma


(tensor([2., 3.]), tensor([2., 3.], device='mps:0'))

### Tensors with gradients

In [50]:
v1 = torch.tensor([2, 3.], requires_grad=True)
v2 = torch.tensor([1., 0.])
v_sum = v1 + v2
v_sum
v_res = (v_sum*2).sum()
v_res
v1.grad, v2.grad, v_sum.grad, v_res.grad

/var/folders/7d/zb8kng_12yz5ccnt2sy4v0mr0000gn/T/ipykernel_21205/2809331367.py:7: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:494.)
  v1.grad, v2.grad, v_sum.grad, v_res.grad


(None, None, None, None)

## Neural Networks in Pytorch (NNs)

In [59]:
import torch.nn as nn

l = nn.Linear(2, 5)
v = torch.FloatTensor([[1, 2]])
l(v)
s = nn.Sequential(
    nn.Linear(2, 5),
    nn.ReLU(),
    nn.Linear(5, 20, bias=False),
    nn.ReLU(),
    nn.Linear(20, 10),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Softmax(dim=1),
)
s

Sequential(
  (0): Linear(in_features=2, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=20, bias=False)
  (3): ReLU()
  (4): Linear(in_features=20, out_features=10, bias=True)
  (5): ReLU()
  (6): Dropout(p=0.3, inplace=False)
  (7): Softmax(dim=1)
)

## nn.Module

In [62]:
class OurModule(nn.Module):
    def __init__(self, num_inputs, num_classes, dropout_prob=0.3):
        super(OurModule, self).__init__()
        self.pipe = nn.Sequential(
            nn.Linear(num_inputs, 5),
            nn.ReLU(),
            nn.Linear(5, 20),
            nn.ReLU(),
            nn.Linear(20, num_classes),
            nn.Dropout(p=dropout_prob),
            nn.Softmax(dim=1),
        )
    
    def forward(self, x):
        return self.pipe(x)

model = OurModule(num_inputs=2, num_classes=5)
model(torch.FloatTensor([[4, 9], [5, 8], [4, 9], [5, 8]]))

tensor([[0.2348, 0.1174, 0.1445, 0.2516, 0.2516],
        [0.1983, 0.0822, 0.1408, 0.2401, 0.3386],
        [0.1974, 0.0987, 0.1215, 0.2115, 0.3708],
        [0.1740, 0.1946, 0.1236, 0.2107, 0.2972]], grad_fn=<SoftmaxBackward0>)

### Loss Functions and Optimizers

#### a common blueprint for a training loop

##### for batch_x, batch_y in iterate_batches(data, batch_size=N):
##### >>>>batch_x_t = torch.tensor(batch_x)
##### >>>>batch_y_t = torch.tensor(batch_y)
##### >>>>out_t = model(batch_x_t)
##### >>>>loss_t = loss_function(batch_y_t, out_t)
##### >>>>loss_t.backward()
##### >>>>optimizer.step()
##### >>>>optimizer.zero_grad()

## TensorBoard

In [64]:

import math
from torch.utils.tensorboard.writer import SummaryWriter

if __name__ == "__main__":
    writer = SummaryWriter()
    funcs = {"sin": math.sin, "cos": math.cos, "tan": math.tan}

    for angle in range(-360, 360):
        angle_rad = math.pi * angle / 180
        for name, fun in funcs.items():
            val = fun(angle_rad)
            writer.add_scalar(name, val, angle)
    writer.close()

## GANs

#### This code in part_02_project.py generates atari-like images using a generator and a discriminator nn.